In [ ]:
!pip install streamlit
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=a69edae91cc12cae7b1744753531721c0893a9983a0f1bf5d0b7b5ea2532d9c0
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import cv2
import zipfile
from sklearn.cluster import KMeans

# Load the images
with zipfile.ZipFile("images.zip", "r") as zip_ref:
    zip_ref.extractall("images")

# Load the images and extract their HOG features
images = []
for i in range(1, 1585):
    file_path = f"/content/images/images/{i}.jpg"
    img = cv2.imread(file_path)
    if img is None:
        print(f"Error: Could not load image at {file_path}")
        continue
    img = cv2.resize(img, (64, 64))
    images.append(img)

images = np.array(images)
features = []
hog_descriptor = cv2.HOGDescriptor((64, 64), (16, 16), (8, 8), (8, 8), 9)
for img in images:
    feature = hog_descriptor.compute(img)
    features.append(feature)

features = np.array(features)

# Define the number of clusters
n_clusters = 16

# Define the KMeans model and fit it to the features
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(features)

# Define a function to process an uploaded image and assign it to a cluster
def process_image(img):
    # Load the image and extract its HOG features
    img = cv2.resize(img, (64, 64))
    feature = hog_descriptor.compute(img)
    
    # Assign the new image to a cluster using nearest neighbor
    closest_cluster = kmeans.predict(feature.reshape(1, -1))[0]
    
    # Update the cluster parameters based on the newly assigned image
    kmeans.cluster_centers_[closest_cluster] = (kmeans.cluster_centers_[closest_cluster] + feature) / 2
    
    return closest_cluster

# Define the Streamlit app
def app():
    # Set the title of the app
    st.set_page_config(page_title="Image Clustering App", page_icon=":smiley:")

    # Create a file uploader and display the uploaded image
    st.sidebar.title("Upload Image")
    uploaded_file = st.sidebar.file_uploader("", type=["jpg", "jpeg", "png"])
    if uploaded_file is not None:
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
        st.image(img, caption="Uploaded Image", use_column_width=True)

        # Process the uploaded image and display the assigned cluster
        cluster = process_image(img)
        st.write(f"Assigned Cluster: {cluster} of 16")

app()        

Writing app.py


In [ ]:
!streamlit run /usr/local/lib/python3.9/dist-packages/ipykernel_launcher.py [ARGUMENTS] --server.port 8501

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: /usr/local/lib/python3.9/dist-packages/ipykernel_launcher.py


In [ ]:
! ngrok config add-authtoken 2Nyjv9orURsVaMXdtKLS2NeVBht_7P4oj1HfwPGakUXET722Q

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:

from pyngrok import ngrok
ngrok.set_auth_token("2OIxEFfLQx4YorIvrljJFSSzJkI_B1tC3LMdawHtJwzCeFPN")

In [ ]:
! nohup streamlit run app.py --server.port 8000 &


nohup: appending output to 'nohup.out'


In [ ]:
from pyngrok import ngrok

# Set the port number for the tunnel
port = 8000

# Define the configuration for the tunnel
config = {
    "addr": port,
    "proto": "http",
    "inspect": True
}

# Start the tunnel using the configuration
public_url = ngrok.connect(**config)

# Print the public URL for the tunnel
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://b85e-34-125-168-218.ngrok-free.app" -> "http://localhost:8000"
